In [1]:
m = 4
n = 3

3

In [2]:
M = [i for i in 1:m]
F = [j for j in 1:n]

3-element Array{Int64,1}:
 1
 2
 3

In [3]:
using Matching
function mat2vecs{T<:Integer}(prefs::Matrix{T})
    return [prefs[1:findfirst(prefs[:, j], 0)-1, j] for j in 1:size(prefs, 2)]
end

mat2vecs (generic function with 1 method)

In [4]:
m_prefs, f_prefs = mat2vecs.(random_prefs(m, n, allow_unmatched=false))
m_prefs, f_prefs = mat2vecs.(random_prefs(m, n))

(Array{Int64,1}[[1],[1,2],[1,2,3],[3,1]],Array{Int64,1}[[1,3,4,2],[3,4],[1,3,4,2]])

In [5]:
for o in 1:m
    push!(m_prefs[o],0)
end

for k in 1:n
    push!(f_prefs[k],0)
end

In [6]:
m_prefs

4-element Array{Array{Int64,1},1}:
 [1,0]    
 [1,2,0]  
 [1,2,3,0]
 [3,1,0]  

In [7]:
f_prefs

3-element Array{Array{Int64,1},1}:
 [1,3,4,2,0]
 [3,4,0]    
 [1,3,4,2,0]

In [8]:
matched_m = zeros(m) 
matched_f = zeros(n)

3-element Array{Float64,1}:
 0.0
 0.0
 0.0

In [9]:
function matching(m_prefs,f_prefs)
    
    M = [g for g in 1:m]
    F = [j for j in 1:n]
    matched_m = zeros(Int64,m)  
    matched_f = zeros(Int64,n)
    accepted_m = zeros(Int64,m)         #男性iの受け入れ先が決まったらaccepted_m[1]=1
    Matrix = zeros(Int64,m,n)
    
    
    while sum(accepted_m) !== m       #男性が全員安定した状況になるまで
                
                for i in 1:m
                    
                    if m_prefs[i][1] == 0       #男性gの第一志望が独身のとき
                        
                        accepted_m[i] = 1
                        matched_m[i] = 0
                
                    end
                    
                    if m_prefs[i][1] !== 0      #男性gに第一志望の女性がいるとき
                        
                        if findfirst(f_prefs[m_prefs[i][1]],i) == 0       #男性gが第一志望の女性の眼中にないとき
                            
                            shift!(m_prefs[i]) #男性iの志望から第一志望を除く
                            
                        end
                        
                        if findfirst(f_prefs[m_prefs[i][1]],i) !== 0      #男性gが第一志望の女性の選択肢に入っているとき
                        
                            if sum(Matrix[:,m_prefs[i][1]]) == 0      #第一志望の女性の列に誰もいないとき
                            
                                matched_m[i] = m_prefs[i][1]
                                matched_f[m_prefs[i][1]] = i
                                Matrix[i,m_prefs[i][1]] = i
                                accepted_m[i] = 1                     
                            
                            end
                        
                            if sum(Matrix[:,m_prefs[i][1]]) !== 0     #第一志望の女性の列に誰かいるとき
                             
                                if findfirst(f_prefs[m_prefs[i][1]],i) < findfirst(f_prefs[m_prefs[i][1]],sum(Matrix[:,m_prefs[i][1]]))
                                   #男性iが保留されていた男性よりも女性の好ましい選択であるとき
                                    
                                    matched_m[i] = m_prefs[i][1]
                                    matched_f[m_prefs[i][1]] = i
                                    Matrix[i,m_prefs[i][1]] = i
                                    Matrix[sum(Matrix[:,m_prefs[i][1]]),m_prefs[i][1]] = 0
                                    shift!(m_prefs[sum(Matrix[:,m_prefs[i][1]])]) 　　#保留されていた男性の志望から第一志望を除く
                                    accepted_m[i] = 1
                                    accepted_m[sum(Matrix[:,m_prefs[i][1]])] = 0
                            
                                end
                                
                                if findfirst(f_prefs[m_prefs[i][1]],i) > findfirst(f_prefs[m_prefs[i][1]],sum(Matrix[:,m_prefs[i][1]]))
                                    #保留されていた男性が男性gよりも女性の好ましい選択であるとき
                                    
                                    shift!(m_prefs[i]) #男性iの志望から第一志望を除く
                                    
                                end
                                
                            end
                            
                        end
                        
                    end
                    
                end
                
            end
            
            return matched_m, matched_f ,Matrix
            
        end

matching (generic function with 1 method)

In [10]:
Pkg.test("MyMatching")

INFO: Testing MyMatching
=============================[ ERROR: MyMatching ]==============================

failed process: Process(`'C:\Users\Masashi Takahashi\AppData\Local\Julia-0.5.1\bin\julia' -Cx86-64 '-JC:\Users\Masashi Takahashi\AppData\Local\Julia-0.5.1\lib\julia\sys.dll' --compile=yes --depwarn=yes --check-bounds=yes --code-coverage=none --color=yes --compilecache=yes 'C:\Users\Masashi Takahashi\.julia\v0.5\MyMatching\test\runtests.jl'`, ProcessExited(1)) [1]



LoadError: MyMatching had test errors

In [11]:
matching(m_prefs,f_prefs)

LoadError: BoundsError: attempt to access 3-element Array{Array{Int64,1},1} at index [0]